In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.

In [2]:
excel_path = "/Users/jakobbengtsson/Desktop/testdata_exjobb.xlsx"
indata_sheet = "indata"
indataDF = pd.read_excel(excel_path,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev
0,1,RDC,Johannesburg,10,40,1,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,Normal,10.0,4.0
2,3,Dealer,Deal2,3,10,1,0.95,Poisson,4.0,4.0
3,4,Dealer,Deal3,2,10,1,0.95,Poisson,4.0,4.0
4,5,Dealer,Deal4,4,10,1,0.95,NBD,3.0,6.0
5,6,Dealer,Deal5,5,10,1,0.95,NBD,3.0,4.0


In [3]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))

In [4]:

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size/L_wh

outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Lead time demand mean,Lead time demand stdev
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,24.0,2.905533,240.0,29.055325
1,2,Dealer,Deal1,10,10,1,0.95,Normal,10.0,4.000000,NaN,NaN
2,3,Dealer,Deal2,3,10,1,0.95,Poisson,4.0,4.000000,NaN,NaN
3,4,Dealer,Deal3,2,10,1,0.95,Poisson,4.0,4.000000,NaN,NaN
4,5,Dealer,Deal4,4,10,1,0.95,NBD,3.0,6.000000,NaN,NaN
5,6,Dealer,Deal5,5,10,1,0.95,NBD,3.0,4.000000,NaN,NaN


RDC reorder-point optimization

In [5]:
h_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Holding cost").to_numpy()
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy()
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy()
mu_wh = mu_L/L_wh * Q_subbatch_size

beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))

beta_arr = np.array(beta_list)

beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
print(f"Optimal weighted induced backorder cost at the warehouse is: {beta_rdc}, betas are: {beta_arr}")



Optimal weighted induced backorder cost at the warehouse is: 0.4936985758902127, betas are: [0.12595016 0.61031741 0.75347723 1.08022691 0.63113497]


In [6]:
h_rdc = float(indataDF.get(indataDF["Type"] == "RDC").get("Holding cost"))
Q_0 = int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))

R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)
outdataDF.loc[outdataDF["Type"] == "RDC","R optimal"] = R_0
outdataDF


Starting optimizing, R = 0, c = 72.60297208217463, c+1 = 74.67909461353156
Doing downwards optimizing, R = -1, c = 70.90774800722352, c+1 = 72.60297208217463
Doing downwards optimizing, R = -2, c = 69.59342238867819, c+1 = 70.90774800722352
Doing downwards optimizing, R = -3, c = 68.69928312250214, c+1 = 69.59342238867819
Doing downwards optimizing, R = -4, c = 68.25618636968849, c+1 = 68.69928312250214
Doing downwards optimizing, R = -5, c = 68.28655655626002, c+1 = 68.25618636968849


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Lead time demand mean,Lead time demand stdev,R optimal
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,24.0,2.905533,240.0,29.055325,-4.0
1,2,Dealer,Deal1,10,10,1,0.95,Normal,10.0,4.000000,NaN,NaN,NaN
2,3,Dealer,Deal2,3,10,1,0.95,Poisson,4.0,4.000000,NaN,NaN,NaN
3,4,Dealer,Deal3,2,10,1,0.95,Poisson,4.0,4.000000,NaN,NaN,NaN
4,5,Dealer,Deal4,4,10,1,0.95,NBD,3.0,6.000000,NaN,NaN,NaN
5,6,Dealer,Deal5,5,10,1,0.95,NBD,3.0,4.000000,NaN,NaN,NaN


In [7]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] =outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Lead time demand mean,Lead time demand stdev,R optimal,Waiting time,Lead time
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,24.0,2.905533,240.000000,29.055325,-4.0,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,Normal,10.0,4.000000,135.708253,54.283301,NaN,3.570825,13.570825
2,3,Dealer,Deal2,3,10,1,0.95,Poisson,4.0,4.000000,26.283301,26.283301,NaN,3.570825,6.570825
3,4,Dealer,Deal3,2,10,1,0.95,Poisson,4.0,4.000000,22.283301,22.283301,NaN,3.570825,5.570825
4,5,Dealer,Deal4,4,10,1,0.95,NBD,3.0,6.000000,22.712476,45.424952,NaN,3.570825,7.570825
5,6,Dealer,Deal5,5,10,1,0.95,NBD,3.0,4.000000,25.712476,34.283301,NaN,3.570825,8.570825


In [16]:
opt_dealer_list = []
L_dealer_arr = outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time")
for Q,L_est,fill_rate_target,demand_type,mu,sigma in zip(Q_dealer_arr,L_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr):
    print(demand_type, mu, sigma)
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,mu,demand_variance = math.pow(sigma,2)))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[1])
    fill_rate_dealer_list.append(tup[2])
    exp_stock_on_hand_list.append(tup[3])

outdataDF.loc[outdataDF["Type"] == "Dealer", "R optimal"] = R_opt_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_list

Normal 10.0 4.0
Poisson 4.0 4.0
Poisson 4.0 4.0
NBD 3.0 6.0
NBD 3.0 4.0


In [17]:
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Lead time demand mean,Lead time demand stdev,R optimal,Waiting time,Lead time,Realized item fill rate,Expected stock on hand
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,24.0,2.905533,240.000000,29.055325,-4.0,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,Normal,10.0,4.000000,135.708253,54.283301,10.0,3.570825,13.570825,0.953944,None
2,3,Dealer,Deal2,3,10,1,0.95,Poisson,4.0,4.000000,26.283301,26.283301,10.0,3.570825,6.570825,0.962711,10.32978
3,4,Dealer,Deal3,2,10,1,0.95,Poisson,4.0,4.000000,22.283301,22.283301,10.0,3.570825,5.570825,0.957439,9.340911
4,5,Dealer,Deal4,4,10,1,0.95,NBD,3.0,6.000000,22.712476,45.424952,10.0,3.570825,7.570825,0.950129,41.149314
5,6,Dealer,Deal5,5,10,1,0.95,NBD,3.0,4.000000,25.712476,34.283301,10.0,3.570825,8.570825,0.952675,26.023837
